<a href="https://colab.research.google.com/github/byeeun/bye/blob/main/%EB%B2%95%EB%AC%B4%EB%B6%80%EC%99%80%20%EA%B2%80%EC%B0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 환경 준비

!pip install konlpy
import pandas as pd

from IPython.display import clear_output
clear_output()

print("###준비가 완료되었습니다.###")

###준비가 완료되었습니다.###


In [ ]:
#@title 데이터 불러오기(탭 \t 으로 분리)
구글드라이브ID = '1rTpYIBkZ6y1gAvEdt-K-anJvYrPY06Io'  #@param {type: "string"}
파일저장이름 = '\uAC80\uCC30\uCD1D\uC7A5 60\uC8FC\uB144 \uAE30\uB150\uC0AC.txt'  #@param {type: "string"}

import time
start = time.time() # 시작시간

# 구글 드라이브 연결을 위한 기본 세팅
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 문서 ID로 실제 파일 불러오는 법
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
rawdata_downloaded = drive.CreateFile({'id': 구글드라이브ID})
rawdata_downloaded.GetContentFile(파일저장이름)

print(파일저장이름+"을 정상적으로 가져왔습니다.")

import pandas as pd
# 파일을 DataFrame으로 전환
rawdata = pd.read_csv(파일저장이름, sep="\t", header=0, encoding='utf-8', engine='python')
rawdata.head()

검찰총장 60주년 기념사.txt을 정상적으로 가져왔습니다.


,id,type,year,text
0,1,임채진,2008,"존경하는 김경한 법무부 장관님을 비롯한 내외 귀빈 여러분, 그리고 전국의 검찰 ..."


In [ ]:
#@title 형태소 분석 및 저장(파이썬은 0부터 시작!!!)
텍스트데이터위치 =   3#@param {type: "number"}
타입데이터위치 =   1#@param {type: "number"}
파일저장이름 = '\uAC80\uCC30\uCD1D\uC7A5 60\uC8FC\uB144 \uAE30\uB150\uC0AC.csv'  #@param {type: "string"}

# 분석하기
from itertools import product
import numpy as np
from konlpy.tag import Okt
okt = Okt()

segments  = []
for index, row in rawdata.iterrows():
    textdata = row[텍스트데이터위치]
    type = row[타입데이터위치]
    words = list(okt.pos(textdata))
    words = pd.DataFrame.from_records(words)
    words['type'] = type
    words['count'] = 1      
    segments.append(words)

# 단일 데이터프레임 구조로 변환
framemerge = pd.concat(segments)

# 데이터프레임의 컬럼값 조정
framemerge.columns = ["terms", "pos", "type", "count"]

print("##################################################")
print("타입별 형태소 추출 결과")
print(framemerge)
print("##################################################")

# 데이터 정리하기
dfwordflag = framemerge.groupby(['terms', 'pos', "type"])['count'].sum()

# 단일 데이터프레임 구조로 변환
dfwordflag = pd.DataFrame(dfwordflag)

print("##################################################")
print("형태소+품사+타입을 기준으로 묶어주기 결과")
print(dfwordflag)
print("##################################################")


#csv로 출력
dfwordflag.to_csv(파일저장이름,header='true', quotechar='"', encoding='utf-8')

print("##################################################")
print("데이터를 " + 파일저장이름 + "로 저장 완료")
print("##################################################")


##################################################
타입별 형태소 추출 결과
      terms          pos type  count
0        존경         Noun  임채진      1
1        하는         Verb  임채진      1
2        김경         Noun  임채진      1
3         한         Josa  임채진      1
4       법무부         Noun  임채진      1
...     ...          ...  ...    ...
1702     약속         Noun  임채진      1
1703   드립니다         Verb  임채진      1
1704      .  Punctuation  임채진      1
1705  감사합니다         Verb  임채진      1
1706      .  Punctuation  임채진      1

[1707 rows x 4 columns]
##################################################
##################################################
형태소+품사+타입을 기준으로 묶어주기 결과
                        count
terms pos         type       
!     Punctuation 임채진       3
,     Punctuation 임채진      32
.     Punctuation 임채진      67
60    Number      임채진       1
60년   Number      임채진       2
...                       ...
획기    Noun        임채진       1
효과    Noun        임채진       4
효율    Noun        임채진       2
힘     Noun